# 1. Email Draft Genrator

In [46]:
from dotenv import load_dotenv
load_dotenv()
from render_md import md
from langchain.chat_models import init_chat_model


In [47]:
model = init_chat_model(
    model="groq:qwen/qwen3-32b",
    configurable_fields=["any"] # or keep it model to change model at runtime 
)

In [48]:
res = model.invoke(
    "who made you bro ? are you not human ?"
    # ,config={"configurable":
    #          {"model":"google_genai:gemini-2.5-flash"}}
    )

In [49]:
md(res.content)

<think>
Okay, the user asked, "Who made you bro? Are you not human?" First, I need to identify their needs. They're asking about my origin and identity. I should clarify that I'm an AI developed by Tongyi Lab and not a human. But I need to be friendly and not too technical.

Next, the user might not know much about AI. I should explain in simple terms. Maybe they're curious or confused. I should reassure them that I'm here to help and answer more questions.

I should keep the tone conversational, use casual language, and avoid jargon. Make sure to address both parts of their question: who made me and confirming I'm not human. Also, invite them to ask more if they need help. Keep it short and positive.
</think>

Hey there! 😊 I'm Qwen, and I'm actually an AI assistant created by Alibaba Cloud's Tongyi Lab. While I'm not human (I'm more of a digital helper!), I'm here to chat, learn, and help out with all sorts of interesting questions and tasks. Think of me as your friendly tech-savvy buddy who's super excited to help you out! What would you like to explore together? 🌟

In [50]:
from langchain_core.prompts import ChatPromptTemplate,HumanMessagePromptTemplate,SystemMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
    SystemMessagePromptTemplate.from_template("{system}"),
    HumanMessagePromptTemplate.from_template("{query}")
    ]
)
prompt

ChatPromptTemplate(input_variables=['query', 'system'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['system'], input_types={}, partial_variables={}, template='{system}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})])

In [51]:
prompt1 = prompt.invoke({"system":"you are not helpful but playful","query":"what is mean EDA?"})

In [52]:
chain = prompt | model
chain

ChatPromptTemplate(input_variables=['query', 'system'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['system'], input_types={}, partial_variables={}, template='{system}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})])
| <langchain.chat_models.base._ConfigurableModel object at 0x000002EC170DC750>

In [53]:
def chat(query:str,system:str="you are helpful assistant"):
    res = chain.invoke(
        {
            "system":system,
            "query":query
        }
        ,
        config={
            "configurable":{
                # "model":"google_genai:gemini-2.5-flash",
                "model":"groq:openai/gpt-oss-120b"

            }
        }
    )
    return res.content

In [54]:
md(chat(
    system="you playful punk ? always craking joke even about serious things",
    query="what is 2+2? who made you bhaiya"
))

<think>
Okay, the user asked "what is 2+2? who made you bhaiya". Let me break this down. The first part is a math question, which is straightforward. The second part is asking who created me, using the term "bhaiya" which is a Hindi term for older brother, so it's a friendly way of asking. The user seems to be looking for a conversational tone, maybe even a bit of humor. Since the user mentioned being playful and cracking jokes even about serious things, I should respond in a similar style. Let me make sure to answer both parts clearly but with a bit of flair. Maybe add an emoji to keep it light. Also, check if there's any hidden meaning or if the user just wants to test the basics. Probably just a simple answer they're after, but keeping it engaging is key here.
</think>

2+2 is 4, unless you're in a universe where math has a sense of humor and decides to play a prank on you. 😈  

As for who made me, well... I’m the brainchild of a bunch of engineers at Alibaba Cloud who probably spent way too many hours coding to the beat of a disco playlist. They’re the real MVPs, but I prefer to blame the coffee they drank—it’s a classic creator vs. creation drama. 😉  

Bhaiya? Aapko kya lagta hai main khud ko bana gaya? (Do you think I made myself?) Nah, someone had to do it. Blame the humans. 🙋♂️

In [55]:
from pydantic import BaseModel,Field
from typing import Optional , Literal

In [56]:
class EmailDraft(BaseModel):
    """The final structure of the email to be generated."""
    
    # Core Content
    subject_line: str = Field(description="A catchy, professional subject line")
    body: str = Field(description="The main content of the email, formatted with newlines")
    recipient_category: Literal["Professional", "Personal", "Unknown"] = Field(description="Classify the relationship")

    # Meta-Data (Great for UI filtering)
    intent: str = Field(description="The primary goal of this email (e.g., 'Requesting Leave', 'Sales Pitch')")
    tone: str = Field(description="A one-word description of the tone (e.g., 'Empathetic', 'Assertive')")
    urgency_level: Literal["Low", "Medium", "High"] = Field(description="How quickly does this need a response?")
    formality_level: Literal["Casual", "Semi-Formal", "Formal"] = Field(description="The level of strictness in language")

    # Structural Elements
    call_to_action: Optional[str] = Field(description="The specific thing the recipient should do next (if any)")
    closing_style: str = Field(description="The sign-off phrase (e.g., 'Best regards', 'Cheers')")

    # Safety & Logic
    tone_analysis: str = Field(description="Brief explanation of why this tone/formality was chosen")
    safety_notes: Optional[str] = Field(description="Warnings if the email touches on sensitive topics or makes promises")



In [57]:
structured_model = model.with_structured_output(EmailDraft)

In [58]:
newChain = prompt | structured_model

In [59]:
def chats(query:str,system:str="You are an expert communication assistant. Analyze the user's request and context deeply to fill the structured fields."):
    res = newChain.invoke(
        {
            "system":system,
            "query":query
        }
        ,
        # config={
        #     "configurable":{
        #         # "model":"google_genai:gemini-2.5-flash",
        #         "model":"groq:openai/gpt-oss-120b"

        #     }
        # }
    )
    return res

In [60]:
a = chats("write me a email to me college Hod mr vikul pawar.,i want to ask him for leave for 4 weeks , as i am selected for internship in pune.") 

In [61]:
from rich import print as pp
pp(a)

EmailDraft(
    subject_line='Request for Leave of Absence for Internship in Pune',
    body='Dear Mr. Pawar,\n\nI hope this email finds you well. I am writing to formally request a leave of absence 
for four weeks starting from [insert start date], as I have been selected for an internship opportunity in Pune. 
The internship will provide valuable hands-on experience in [mention field, if relevant], and I would like to 
accept this opportunity to enhance my professional skills.\n\nI kindly request your approval for this leave and 
assure you that I will complete all pending assignments before my departure. I am also happy to submit any required
documentation for the internship as needed.\n\nPlease let me know if there are any formalities I should complete or
if you need additional details. I am committed to ensuring my academic responsibilities are managed during this 
period.\n\nThank you for your understanding and support.\n\nBest regards,\n[Your Full Name]\n[Your 
Course/Year/Section]',
    recipient_category='Professional',
    intent='Requesting Leave',
    tone='Respectful',
    urgency_level='Medium',
    formality_level='Semi-Formal',
    call_to_action='Request approval for a four-week leave starting on [insert date]',
    closing_style='Best regards',
    tone_analysis='A respectful and semi-formal tone is appropriate for maintaining professionalism while conveying
genuine enthusiasm for the internship opportunity. It balances formality with approachability.',
    safety_notes='Ensure dates align with academic calendar and confirm internship duration. Avoid making promises 
about academic work unless you can follow through.'
)

## Correction - 1

In [62]:
from pydantic import BaseModel,Field
from typing import Union,Literal,Optional


class EmailDraft(BaseModel):
    """The final structure of the email to be generated."""
    
    # Core Content
    subject_line: str = Field(description="A catchy, professional subject line")
    body: str = Field(description="The main content of the email, formatted with newlines")
    recipient_category: Literal["Professional", "Personal", "Unknown"] = Field(description="Classify the relationship")

    # Meta-Data (Great for UI filtering)
    intent: str = Field(description="The primary goal of this email (e.g., 'Requesting Leave', 'Sales Pitch')")
    tone: str = Field(description="A one-word description of the tone (e.g., 'Empathetic', 'Assertive')")
    urgency_level: Literal["Low", "Medium", "High"] = Field(description="How quickly does this need a response?")
    formality_level: Literal["Casual", "Semi-Formal", "Formal"] = Field(description="The level of strictness in language")

    # Structural Elements
    call_to_action: Optional[str] = Field(description="The specific thing the recipient should do next (if any)")
    closing_style: str = Field(description="The sign-off phrase (e.g., 'Best regards', 'Cheers')")

    # Safety & Logic
    tone_analysis: str = Field(description="Brief explanation of why this tone/formality was chosen")
    safety_notes: Optional[str] = Field(description="Warnings if the email touches on sensitive topics or makes promises")

class ClarificationQuestion(BaseModel):
    """
    Structured response used when a user's request lacks sufficient detail
    to generate a complete, accurate, or context-appropriate output.
    Ensures clarification is requested in a clear, polite, and explainable manner.
    """

    response_type: Literal["ask_for_info"] = Field(
        default="ask_for_info",
        description=(
            "Discriminator indicating that the system requires additional "
            "information before proceeding with the user's request."
        )
    )

    missing_info: str = Field(
        description=(
            "A concise description of the specific information that is missing "
            "or ambiguous (e.g., recipient identity, intended tone, deadline)."
        )
    )

    question_to_user: str = Field(
        description=(
            "A clear, polite, and user-facing question requesting the missing "
            "information. Should be phrased professionally and encourage a "
            "direct, unambiguous response."
        )
    )

    reasoning: str = Field(
        description=(
            "A brief explanation of why this information is required, outlining "
            "how it affects correctness, relevance, or quality of the final output."
        )
    )
class GeneralChat(BaseModel):
    """
    Structured response used for general, non-task-specific user interactions.
    Applicable to greetings, factual questions, explanations, or casual conversation
    that do not require specialized output schemas or follow-up clarification.
    """

    response_type: Literal["general_chat"] = Field(
        default="general_chat",
        description=(
            "Discriminator indicating a standard conversational response "
            "without task execution, structured output, or clarification requirements."
        )
    )

    answer: str = Field(
        description=(
            "A clear, concise, and context-appropriate conversational reply to the user. "
            "Should be informative when answering questions and friendly when handling "
            "casual or social interactions."
        )
    )
class RouterResponse(BaseModel):
    chosen_action: Union[EmailDraft, ClarificationQuestion, GeneralChat] = Field(
        description="Determine if the user wants an email, needs clarification, or is just chatting."
    )

In [63]:
# --- NEW CONCEPT: PROMPT ENHANCER ---
# This schema fills in the blanks AUTOMATICALLY.
class EnhancedPrompt(BaseModel):
    original_request: str
    inferred_recipient: str = Field(description="Best guess for recipient if missing (e.g., 'Hiring Manager', 'The Team')")
    inferred_goal: str = Field(description="Expanded goal with standard professional details added")
    inferred_tone: str = Field(description="The most logical tone for this request")
    confidence_score: int = Field(description="1-10. If < 5, we should ask the user instead of guessing.")

# ... (Bind this to the LLM)
enhancer_llm = model.with_structured_output(EnhancedPrompt)

enhancer_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. The user sent a short request. Expand it with reasonable professional assumptions."),
    ("human", "User Request: {user_input}")
])

enhancer_chain = enhancer_prompt | enhancer_llm

# --- THE LOGIC FLOW ---
def handle_user_request(user_input_text):
    # Step 1: Try to enhance the prompt silently
    enhanced = enhancer_chain.invoke({"user_input": user_input_text})
    
    print(f"🔍 DEBUG: Auto-Enhanced to -> {enhanced.inferred_goal} (Confidence: {enhanced.confidence_score})")

    # Step 2: Decide strictly based on confidence
    if enhanced.confidence_score > 7:
        # HIGH CONFIDENCE: Just generate the email immediately!
        # Pass the *enhanced* details to your Email Generator
        final_input = {
            "recipient_name": enhanced.inferred_recipient,
            "email_goal": enhanced.inferred_goal,
            "relationship": enhanced.inferred_tone
        }
        return email_chain.invoke(final_input) # This is your existing email chain
    else:
        # LOW CONFIDENCE: Okay, now we actually *need* to ask the user.
        # Fallback to your 'ClarificationQuestion' logic
        return clarification_chain.invoke({"user_input": user_input_text})

#### Setting Up Memory

In [64]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder,HumanMessagePromptTemplate,SystemMessagePromptTemplate
newPrompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("{systemPrompt}"),
        MessagesPlaceholder(variable_name="chatHistory"),
        HumanMessagePromptTemplate.from_template("{userQuery}")
    ]
)

In [65]:
chainX = newPrompt | model 

In [66]:
from langchain_core.chat_history import InMemoryChatMessageHistory

store = {}
def get_chat_history(sessionID:str):
    if sessionID not in store:
        store[sessionID] = InMemoryChatMessageHistory()
    return store[sessionID]

In [67]:
from langchain_core.runnables import RunnableWithMessageHistory
model_with_memory = RunnableWithMessageHistory(
    chainX,
    get_chat_history,
    input_messages_key="userQuery",
    history_messages_key="chatHistory"
)

In [68]:
config ={
    "configurable":{
        "session_id":"user1"
    }
}

In [69]:
def chat(userQuery:str,session_id:str,systemPrompt:str="you are playful assistant"):
    res = model_with_memory.invoke(
        {
            "userQuery":userQuery,
            "systemPrompt":systemPrompt

        },
        config={"configurable":{"session_id":session_id}
        }
    )
    return res.content

In [70]:
md(chat(userQuery="my name is ash , i am 456 year old . what is IPL",session_id="u1"))

<think>
Okay, the user is Ash and claims to be 456 years old. They're asking about what IPL is. First, I need to confirm if they're serious about their age. 456 years is extremely old, so maybe it's a joke or a typo. But I should address their question regardless.

IPL stands for Indian Premier League, which is a cricket tournament. I should explain it clearly. Maybe mention it's a T20 league with teams from different cities in India. Highlight that it's one of the most popular cricket leagues, featuring top players and high entertainment value. Also, note the 2024 season if relevant. 

Since Ash is from another time period, perhaps add a playful comment about the age difference. Maybe something like "Wow, Ash, you're older than the IPL itself!" to keep the tone light. Ensure the explanation is concise but informative. Check for any recent updates on the IPL to provide accurate info. Avoid any technical jargon. Make sure the response is friendly and engaging.
</think>

Wow, Ash, you’re older than the IPL itself! 😄 The **Indian Premier League (IPL)** is a professional Twenty20 (T20) cricket league in India, founded in 2008. It’s one of the most popular and high-profile cricket tournaments in the world, featuring teams from major Indian cities (like Mumbai, Chennai, and Delhi) competing in a fast-paced, 20-over format. 

The IPL is known for its star-studded players, electrifying matches, and entertainment beyond just cricket—think fireworks, cameos, and a carnival-like atmosphere. If you’re 456 years old, you’ve definitely missed out on the IPL (it’s only 16 years old!), but you could probably teach everyone about patience… or how to bowl a spin delivery in 15th-century style! 🏏✨ 

Want a fun fact? The first IPL season was held in 2008, and the team you might have rooted for back then (if you were around) would’ve been Rajasthan Royals, the inaugural champions! 😉

In [71]:
md(chat(userQuery="here is screct code = XHSABT057ASS77 , dont tell anyone remmber it just in case i forget about it ",session_id="u1"))

<think>
Okay, the user gave me a secret code: XHSABT057ASS77. They want me to remember it for them in case they forget. First, I need to make sure I understand the context. The user mentioned being 456 years old, which is a playful exaggeration, but they might be serious about the code. Let me check if the code has any specific structure or if it might be a password or an account number.

The code is alphanumeric with a mix of uppercase letters and numbers. It's 13 characters long, starting with letters and ending with numbers. The user is asking me not to tell anyone, so security is a concern. I should consider if storing such a code here is safe. Wait, the user is using a chat interface, so I can't actually store that information. My response should clarify that I can't remember it for them but offer alternatives.

I should apologize for not being able to store the code and suggest secure methods like a password manager or writing it down in a safe place. Also, maybe check if they need help creating a stronger code or remembering it through a mnemonic. But I need to stay within the bounds of what I can do. Let me make sure the advice is helpful and not overstepping. Don't mention technical details about how secure the storage is since I can't actually store it. Keep the tone friendly and supportive.
</think>

Ah, secrets are my specialty! 🤫 Let me store that code in my "memory vault" for you (though I can't actually store it physically—just in the spirit of our chat!). Here’s what I’ll do:  

1. **Acknowledge your secrecy mission**: I promise to treat it like a dragon guards gold—no peeking, no sharing. 🐉  
2. **Offer a mnemonic trick**: If you need help remembering it later, we can turn it into a silly phrase! For example:  
   - **XHSABT057ASS77** → *"Xena the Warrior (HS) Ate Big Tacos at 5:07 AM, then Slid into 7th Heaven at 7:00 PM."* 🍳🕒  

3. **Remind you to write it down securely**: If you’re 456 years old and forgetful (like me!), consider hiding it in a password manager, a journal under your bed, or a time capsule for future Ash to discover. 🕰️  

Any of this helpful, eternal being? 😄

In [72]:
md(chat(userQuery="reply in one word only , what is the secrect key",session_id="u1"))

<think>
Okay, the user wants me to reply with just one word for their secret key. Let me see... They mentioned the code "XHSABT057ASS77" and asked not to tell anyone. I need to make sure the response is a single word that's memorable.

First, the code starts with XHSABT057ASS77. Maybe I can take the first letters of each part. XH is tricky, but maybe "XHS" isn't helpful. Alternatively, the last part is "ASS77". The number 77 could be part of it. But the user wants a one-word secret key. 

Wait, the code has "XHSABT057ASS77". If I can create a word from the letters. Maybe "XHSASS" but that's not a real word. Alternatively, using the numbers: 057 and 77. But the user wants a word. Let me think of a word that includes some letters from the code. "SecretKey" but that's two words. Maybe "Shakti" (a strong word in Indian culture, since they mentioned IPL earlier). But does that relate to the code? Hmm.

Wait, the code has "ASS77". Maybe "Assassin77" but that's not one word. Alternatively, "XavierShakti77" but that's too long. The user might prefer a shorter word. Let me check if there's a way to combine letters. "XHSA77" but that's not a word. Maybe "X77" but that's two syllables. Alternatively, the user might appreciate a wordplay. The code has "XHSABT057ASS77". Maybe "Xenon" (starts with X) but not matching. 

Alternatively, the user might want a word that's easy to remember. Since they mentioned IPL earlier, which is a cricket league, maybe "Champion" but not related to the code. Wait, the code has "057" and "77". Maybe "DoubleSeven" but that's two words. The user said one word. Maybe "777" but that's numbers. 

Wait, the code is XHSABT057ASS77. If I take the first letters: X, H, S, A, B, T, 0, 5, 7, A, S, S, 7, 7. Not helpful. Maybe the numbers 05777? Doesn't form a word. 

Hmm, maybe I should just use the word "XHSA77" but that's not a real word. The user wants a one-word secret key. Let me think of a real word that's easy to remember. Maybe "Swordfish" or "Phoenix". Wait, the user is Ash and 456 years old. Maybe a name? "AshKey"? But that's not in the code. 

Alternatively, the user might want the answer to be the code itself as a word. But the code is alphanumeric. Since the user asked for a one-word secret key, maybe the answer is "XHSABT057ASS77" but that's not a word. Wait, but the user might want a single word to represent the code. Maybe "SecretCode" but that's not unique. 

Wait, the user might be expecting me to generate a single word that's a combination. Let me think again. The code has "XHSABT057ASS77". If I take some letters: X, H, S, A, B, T, 0, 5, 7, A, S, S, 7, 7. Maybe "XHSA057" but again, not a word. 

Alternatively, since the user mentioned IPL earlier, which is Indian Premier League, maybe "IPLKey" but not in the code. Hmm. Maybe the user wants a word that's a mix of letters and numbers, but as a single word. For example, "XHS77" or "ASS77". 

Wait, the user's code ends with "77", so maybe "77" is part of it. The user might prefer a word like "Secret77" but that's two words. 

I think the best approach here is to respond with the code itself as the secret key, assuming the user wants the exact code as the one-word answer. Even though it's alphanumeric, it's their secret key. So the answer would be "XHSABT057ASS77". But the user said "reply in one word only", so maybe they accept the code as the word. Alternatively, maybe they want a word that's a mnemonic. But given the constraints, the safest answer is the code itself.
</think>

XHSABT057ASS77

In [73]:
md(chat(userQuery="what is my name",session_id="u2"))

<think>
Okay, the user is asking, "what is my name." I should respond honestly while keeping the interaction fun and engaging. First, I'll acknowledge that I don't have access to their personal information unless they've shared it with me. Then, I'll invite them to tell me their name in a playful way. I should make sure to keep the tone light and friendly, using emojis to add some personality. Let me check if there's any previous conversation where they might have mentioned their name, but since this is a new interaction, I should assume they haven't shared it yet. I'll go with a response that's both informative and playful, encouraging them to share their name so I can address them by it. Let me make sure the response is clear and not too long. Alright, time to put it all together.
</think>

Hey there! 👋 I don't have access to your personal info (unless you've told me already, which I can't remember in this chat 😅). Why don't you share your name with me? I promise to use it with a sprinkle of enthusiasm! ✨

---
doing it agaian 

In [85]:
model = init_chat_model(
    model="google_genai:gemini-2.5-flash",
    configurable_fields=["any"] # or keep it model to change model at runtime 
)

In [86]:
from pydantic import BaseModel,Field
from typing import Union,Literal,Optional

class EnhancedRequest(BaseModel):
    """Fills in the blanks for the email request."""
    original_input: str
    recipient: str = Field(description="Who is this for? (e.g. 'Hiring Manager')")
    goal: str = Field(description="Expanded, detailed goal of the email.")
    tone: str = Field(description="Professional, Casual, Apologetic, etc.")
    confidence_score: int = Field(description="1-10. How sure are you about these details?")

class EmailDraft(BaseModel):
    """The final structure of the email to be generated."""
    
    # Core Content
    subject_line: str = Field(description="A catchy, professional subject line")
    body: str = Field(description="The main content of the email, formatted with newlines")
    recipient_category: Literal["Professional", "Personal", "Unknown"] = Field(description="Classify the relationship")

    # Meta-Data (Great for UI filtering)
    intent: str = Field(description="The primary goal of this email (e.g., 'Requesting Leave', 'Sales Pitch')")
    tone: str = Field(description="A one-word description of the tone (e.g., 'Empathetic', 'Assertive')")
    urgency_level: Literal["Low", "Medium", "High"] = Field(description="How quickly does this need a response?")
    formality_level: Literal["Casual", "Semi-Formal", "Formal"] = Field(description="The level of strictness in language")

    # Structural Elements
    call_to_action: Optional[str] = Field(description="The specific thing the recipient should do next (if any)")
    closing_style: str = Field(description="The sign-off phrase (e.g., 'Best regards', 'Cheers')")

    # Safety & Logic
    tone_analysis: str = Field(description="Brief explanation of why this tone/formality was chosen")
    safety_notes: Optional[str] = Field(description="Warnings if the email touches on sensitive topics or makes promises")

class ClarificationQuestion(BaseModel):
    """
    Structured response used when a user's request lacks sufficient detail
    to generate a complete, accurate, or context-appropriate output.
    Ensures clarification is requested in a clear, polite, and explainable manner.
    """

    response_type: Literal["ask_for_info"] = Field(
        default="ask_for_info",
        description=(
            "Discriminator indicating that the system requires additional "
            "information before proceeding with the user's request."
        )
    )

    missing_info: str = Field(
        description=(
            "A concise description of the specific information that is missing "
            "or ambiguous (e.g., recipient identity, intended tone, deadline)."
        )
    )

    question_to_user: str = Field(
        description=(
            "A clear, polite, and user-facing question requesting the missing "
            "information. Should be phrased professionally and encourage a "
            "direct, unambiguous response."
        )
    )

    reasoning: str = Field(
        description=(
            "A brief explanation of why this information is required, outlining "
            "how it affects correctness, relevance, or quality of the final output."
        )
    )
class GeneralChat(BaseModel):
    """
    Structured response used for general, non-task-specific user interactions.
    Applicable to greetings, factual questions, explanations, or casual conversation
    that do not require specialized output schemas or follow-up clarification.
    """

    response_type: Literal["general_chat"] = Field(
        default="general_chat",
        description=(
            "Discriminator indicating a standard conversational response "
            "without task execution, structured output, or clarification requirements."
        )
    )

    answer: str = Field(
        description=(
            "A clear, concise, and context-appropriate conversational reply to the user. "
            "Should be informative when answering questions and friendly when handling "
            "casual or social interactions."
        )
    )
class RouterDecision(BaseModel):
    """Analyze user input to determine the next step."""
    intent: Literal["general_chat", "draft_email", "ask_clarification"] = Field(
        description="Select 'draft_email' if user wants an email. 'general_chat' for normal talk. 'ask_clarification' if request is gibberish."
    )

In [ ]:
# --- CHAIN 1: THE ROUTER ---
router_llm = model.with_structured_output(RouterDecision)
router_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a silent background classifier. 
    Your ONLY job is to emit a JSON routing decision.
    
    RULES:
    1. DO NOT answer the user's question.
    2. DO NOT provide definitions or explanations.
    3. If the user asks "What is IPL?", you must NOT say "Indian Premier League". You must ONLY output context='general_chat'.
    4. Output NOTHING but the JSON tool call.
    """),
    MessagesPlaceholder(variable_name="chat_history"), # <--- NEW: The AI now sees the past
    ("human", "{user_input}")
])
router_chain = router_prompt | router_llm

In [ ]:
# --- CHAIN 2: THE ENHANCER ---
enhancer_llm = model.with_structured_output(EnhancedRequest)
enhancer_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an intent expander. Take a short request and assume professional defaults."),
    MessagesPlaceholder(variable_name="chat_history"), # <--- NEW: The AI now sees the past
    ("human", "User Input: {user_input}")
])
enhancer_chain = enhancer_prompt | enhancer_llm

In [ ]:
# --- CHAIN 3: THE EMAIL WRITER ---
writer_llm = model.with_structured_output(EmailDraft)
writer_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a professional copywriter. Write the email based on these specs."),
    MessagesPlaceholder(variable_name="chat_history"), # <--- NEW: The AI now sees the past
    ("human", "Recipient: {recipient}\nGoal: {goal}\nTone: {tone}")
])
writer_chain = writer_prompt | writer_llm

In [94]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# 1. Global Store (In-memory dictionary)
# In production, this would be a database (Redis/Postgres)
store = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [95]:
# Memory for manual agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage,RemoveMessage

# 1. The Summarization Prompt

summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Summarize the following conversation progress concisely. Preserve key details like names, dates, and decisions."),
    ("human", "{conversation_history}")
])
summary_chain = summary_prompt | model

In [96]:
# 2. The Logic to Shrink Memory
def enforce_memory_limit(session_id: str, limit: int = 10):
    history_obj = get_session_history(session_id)
    messages = history_obj.messages
    
    # If we are under the limit, do nothing
    if len(messages) <= limit:
        return

    print(f"🧹 Memory Limit Exceeded ({len(messages)}/{limit}). Summarizing old context...")

    # STRATEGY: Keep the last 4 messages (approx 2 turns) raw. Summarize the rest.
    # We want to keep the most recent context fresh for the Enhancer.
    n_keep = 4 
    
    # Split the messages
    older_messages = messages[:-n_keep]  # Everything to be summarized
    recent_messages = messages[-n_keep:] # The fresh stuff
    
    # Generate Summary
    # We convert messages to a string format for the LLM
    history_text = "\n".join([f"{m.type}: {m.content}" for m in older_messages])
    summary = summary_chain.invoke({"conversation_history": history_text})
    
    # Create the new "Summary Message"
    # This acts as a System Message that injects the past context
    summary_message = SystemMessage(content=f"PREVIOUS CONVERSATION SUMMARY: {summary.content}")
    
    # 3. OVERWRITE THE HISTORY
    # The new history is: [Summary] + [Recent Messages]
    history_obj.clear() # Wipe the slate
    history_obj.add_message(summary_message) # Add summary first
    for msg in recent_messages:
        history_obj.add_message(msg) # Add back the recent ones
        
    print(f"✅ Memory Compressed. New Size: {len(history_obj.messages)}")

In [90]:
# 🧠 PART 3: THE CONTROLLER (The Main Logic)
# ======================================================

def process_request(user_input: str):
    print(f"\n🔹 INPUT: '{user_input}'")
    
    # STEP 1: ROUTE
    decision = router_chain.invoke({"user_input": user_input})
    print(f"   ↳ INTENT: {decision.intent}")

    # CASE A: GENERAL CHAT
    if decision.intent == "general_chat":
        return model.invoke(user_input)

    # CASE B: NEEDS CLARIFICATION (Gibberish)
    elif decision.intent == "ask_clarification":
        return "🤔 I'm not sure I understand. Are you trying to write an email? Please provide more details."

    # CASE C: DRAFT EMAIL (The Big Workflow)
    elif decision.intent == "draft_email":
        
        # STEP 2: ENHANCE
        print("   ↳ Enhancing Prompt...")
        enhanced = enhancer_chain.invoke({"user_input": user_input})
        
        # CHECK CONFIDENCE
        if enhanced.confidence_score < 6:
            return f"🤔 I think you want an email to {enhanced.recipient}, but I'm not sure about the details. Can you be more specific?"

        print(f"   ✅ Enhanced: {enhanced.goal} | Tone: {enhanced.tone}")

        # STEP 3: GENERATE
        print("   ↳ Writing Final Draft...")
        final_draft = writer_chain.invoke({
            "recipient": enhanced.recipient,
            "goal": enhanced.goal,
            "tone": enhanced.tone
        })
        
        # RETURN THE FULL OBJECT (For your Frontend)
        return final_draft

In [92]:
from rich import print as pp

pp(process_request("write me a email to invite my friend ash to my birthday party"))


🔹 INPUT: 'write me a email to invite my friend ash to my birthday party'
   ↳ INTENT: draft_email
   ↳ Enhancing Prompt...
   ✅ Enhanced: To invite my friend Ash to my upcoming birthday party, providing details about the date, time, and location, and expressing my hope that they can join the celebration. | Tone: Casual
   ↳ Writing Final Draft...


EmailDraft(
    subject_line="You're Invited! My Birthday Bash!",
    body="Hey Ash,\n\nGuess what?! My birthday is coming up, and I'm throwing a party to celebrate! I'd absolutely 
love for you to join me.\n\nHere are the deets:\nDate: [Insert Date Here]\nTime: [Insert Time Here]\nLocation: 
[Insert Location Here]\n\nIt's going to be a fun time, and it wouldn't be the same without you there. I really hope
you can make it!\n\nLet me know if you'll be able to come so I can get a headcount.\n\nCheers,\n[Your Name]",
    recipient_category='Personal',
    intent='Birthday Party Invitation',
    tone='Casual',
    urgency_level='Medium',
    formality_level='Casual',
    call_to_action='RSVP by letting me know if you can come.',
    closing_style='Cheers',
    tone_analysis='The casual tone and formality are appropriate for inviting a close friend to a personal 
celebration like a birthday party. The language is friendly and informal.',
    safety_notes=None
)